# Operator Definitions
We will define the fermionic creation and annihilation operators $c_j^{\dagger}$ and $c_j$ respectively.

In [37]:
import numpy as np
from dataclasses import dataclass, field
from numpy.typing import NDArray
from scipy.sparse import csr_matrix, identity
from typing import List

In [38]:
Z = np.array([[1, 0], [0, -1]])
creation_operator = np.array([[0, 0], [1, 0]])
annihilation_operator = np.array([[0, 1], [0, 0]])

In [39]:
# Antisymmetry Confirmations for Fermionic Operators 
vacuum_state = np.array([1, 0])
occupied_state = np.array([0, 1])

# c† |0> = |1>
assert((creation_operator @ vacuum_state == occupied_state).all())

# c† |0> = NULL
assert((creation_operator @ occupied_state == np.zeros(2,)).all())

# c |1> = |0>
assert((annihilation_operator @ occupied_state == vacuum_state).all())

# c† |0> = NULL
assert((annihilation_operator @ vacuum_state == csr_matrix(np.zeros(2,))).all())

# c†c + cc† = I
assert((annihilation_operator @ creation_operator + creation_operator @ annihilation_operator == np.eye(2)).all())

In [40]:
def creation_gate(i, N):
    ops = np.eye(2**N)

    for j in range(i):
        Z_j = np.kron(np.eye(2**j), np.kron(Z, np.eye(2**(N-j-1))))
        ops = ops @ Z_j
    
    c_i = np.kron(np.eye(2**i), np.kron(creation_operator, np.eye(2**(N-i-1))))
    ops = ops @ c_i
    
    return csr_matrix(ops)

def annihilation_gate(i, N): 
    ops = np.eye(2**N)

    for j in range(i):
        Z_j = np.kron(np.eye(2**j), np.kron(Z, np.eye(2**(N-j-1))))
        ops = ops @ Z_j
    
    c_i = np.kron(np.eye(2**i), np.kron(annihilation_operator, np.eye(2**(N-i-1))))
    ops = ops @ c_i
    
    return csr_matrix(ops)

In [53]:
# Required Classes
@dataclass 
class KitaevChain: 
    N: int
    creations: List[csr_matrix] = field(init=False)
    annihilations: List[csr_matrix] = field(init=False)
    state_vector : csr_matrix = field(init=False)

    def __post_init__(self):
        # Create and store creation and annihilation operators for each fermionic mode
        self.creations = []
        self.annihilations = []
        

        for i in range(self.N): 
            self.creations.append(creation_gate(i, self.N))
            self.annihilations.append(annihilation_gate(i, self.N))

        self.state_vector = np.zeros(2**self.N,)
        self.state_vector[0] = 1
        self.state_vector = csr_matrix(self.state_vector).T
            
    def __repr__(self): 
        return f'{self.__class__.__name__} with {self.N} fermionic modes'